In [72]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [73]:
%run trips_duration.ipynb

object


/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_81298/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [74]:
def merging_data(final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()

    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    tourist_num.sort_values(by = ["Year"])

    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])

    return tourist_num, unfiltered_predicted_tourist


In [75]:
datas = merging_data(final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data_not_st

,Country,Year,NumTrips
0,Albania,2017,8186685
1,Albania,2019,10176614
2,Albania,2020,6624361
3,Austria,2012,36003256
4,Austria,2013,36479782
...,...,...,...
345,Switzerland,2018,40810491
346,Switzerland,2019,36986339
347,Switzerland,2020,31748407
348,Switzerland,2021,32989206


In [146]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['Bias'] = 1
    
    index_country = country_df.index[country_df['Year'].astype(int) == 2015]
    index = index_country[0]

    y_vector = country_df.iloc[index :]['Tourism St.'].tolist()
    x_matrix = []
    x_list = []
    for i in range(index, len(country_df)): 
        if i - lag < 0:
            continue
        bias = 1 
        country = country_df.iloc[i]["Country"]
        lag_values = [country_df.iloc[i - j]['Tourism St.'] for j in range(1, lag + 1)]
        covid = country_df.iloc[i]['Covid']
        year = country_df.iloc[i]['Year']
        x_list = [bias, country, year, covid] + lag_values
        x_matrix.append(x_list)

    x_matrix = pd.DataFrame(x_matrix)
    y_vector = y_vector[lag:]  


    return x_matrix, y_vector

In [147]:
data[data['Country']=='Slovakia']

,Country,Year,Tourism St.
294,Slovakia,2012,0.021458
295,Slovakia,2013,0.020559
296,Slovakia,2014,0.021513
297,Slovakia,2015,0.025319
298,Slovakia,2016,0.030385
299,Slovakia,2017,0.034212
300,Slovakia,2018,0.039610
301,Slovakia,2019,0.044606
302,Slovakia,2020,0.019206
303,Slovakia,2021,0.023941


In [ ]:
X, y = get_lag_columns(data[data['Country']=='Slovakia'])

(   0         1     2  3         4         5         6
 0  1  Slovakia  2015  0  0.021513  0.020559  0.021458
 1  1  Slovakia  2016  0  0.025319  0.021513  0.020559
 2  1  Slovakia  2017  0  0.030385  0.025319  0.021513
 3  1  Slovakia  2018  0  0.034212  0.030385  0.025319
 4  1  Slovakia  2019  0  0.039610  0.034212  0.030385
 5  1  Slovakia  2020  1  0.044606  0.039610  0.034212
 6  1  Slovakia  2021  1  0.019206  0.044606  0.039610
 7  1  Slovakia  2022  0  0.023941  0.019206  0.044606
 8  1  Slovakia  2023  0  0.041598  0.023941  0.019206,
 [0.0396101720567253,
  0.04460578041262051,
  0.019206153354248967,
  0.023941342883577185,
  0.04159770013749046,
  0.04807679240371502])

In [149]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.inv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [152]:
def predict_next_year(beta, X):
    return np.dot(beta, X)

In [161]:

def get_country_years_pred(data, country, lag = 3):
    X, y = data[data['Country'] == country]
    country_df = get_lag_columns(country_df, lag = lag)
    
    X = country_df[['Bias', 'Tourism_lag']].values
    y = country_df['Tourism St.'].values

    beta_tourism = fit_regression(X, y)

    predictions = []
    for idx, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row['Bias'], row['Tourism_lag']])
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_Tourism': row['Tourism St.'],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [162]:
prev_year_result = get_country_years_pred(data, "Slovenia", lag=3)
print(prev_year_result)


ValueError: too many values to unpack (expected 2)

In [159]:

def get_country_prediction(data, country, lag=3):
    country_df0 = data[data['Country'] == country].copy()
    country_df = get_lag_columns(country_df0, lag=lag)

    X = country_df[['Bias', 'Tourism_lag']].values
    y_tourism = country_df['Tourism St.'].values

    beta_tourism = fit_regression(X, y_tourism)

    # Get the most recent year in the dataset
    last_year = country_df['Year'].max()

    # Get initial lag values
    lag_year = last_year - lag + 1
    lag_row = country_df[country_df['Year'] == lag_year]
    
    tourism_lag_val = lag_row['Tourism St.'].values[0]

    # Store predictions for each future year
    predictions = []

    for i in range(1, lag + 1):
        year = last_year + i
        X_new = np.array([1, tourism_lag_val])
        
        pred_tourism = predict_next_year(beta_tourism, X_new)

        predictions.append({
            'Year': year,
            'Country': country,
            'Predicted_Tourism': pred_tourism
        })

        # Update lag values for next step
        tourism_lag_val = pred_tourism

    return predictions


In [160]:
result = get_country_prediction(data, "Italy", lag=3)
result = pd.DataFrame(result)
result.sort_values(by=['Year'])
result

TypeError: tuple indices must be integers or slices, not list

In [83]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    
    country_name = st_data["Country"].iloc[0]
    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)
    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])

    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Tourism_combined'], 
        name='Tourist Numbers',
        line=dict(color='royalblue', width=4),
    ))

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)

    fig.show()


In [84]:
visualization(data_not_st, result)

In [85]:
def cross_validate_time_series(country_df, lag = 3):
    results = []
    country_df = country_df.sort_values('Year').reset_index(drop = True)

    for i in range(lag, len(country_df) - 1):
        train_df = country_df.iloc[:i]
        test_df = country_df.iloc[i:i+1]

        X_train = train_df[['Bias', 'Tourism_lag']].values
        y_tourism_train = train_df['Tourism St.'].values

        X_test = test_df[['Bias', 'Tourism_lag']].values[0]

        beta_tourism  = fit_regression(X_train, y_tourism_train)

        pred_tourism = predict_next_year(beta_tourism, X_test)

        results.append({
            'Year': test_df['Year'].values[0],
            'Actual_Tourism': test_df['Tourism St.'].values[0],
            'Predicted_Tourism': pred_tourism
        })

    predictions = pd.DataFrame(results)

    return predictions

In [86]:
def evaluate_predictions(pred_df):
    r2_tourism = r2_score(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])
    mse_tourism = mean_squared_error(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])

    evaluations = {
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [87]:
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]

for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
    country_df = get_lag_columns(country_df, lag = 3)
    # country_df

    pred_df = cross_validate_time_series(country_df, lag = 3)
    metrics = evaluate_predictions(pred_df)

    #print(pred_df)
    print(country)
    print(metrics)

Austria
{'Tourism': {'R2': -0.5312821675670367, 'MSE': 0.0006717739605425716}}
Belgium
{'Tourism': {'R2': -0.03399784042694143, 'MSE': 0.00021692867375729873}}
Bulgaria
{'Tourism': {'R2': -0.45772135408116665, 'MSE': 1.9489010942590678e-05}}
Croatia
{'Tourism': {'R2': -0.8207146146209252, 'MSE': 1.3828470061487988e-05}}
Czechia
{'Tourism': {'R2': -0.42525225239617703, 'MSE': 0.000403866853040582}}
Denmark
{'Tourism': {'R2': -5.8879802420882, 'MSE': 0.0014736729488417054}}
Estonia
{'Tourism': {'R2': -0.8148312570219132, 'MSE': 3.129986033743845e-05}}
Finland
{'Tourism': {'R2': -0.42500281421291675, 'MSE': 0.00022814857313834905}}
France
{'Tourism': {'R2': -7.496568887449451, 'MSE': 0.05479377452403642}}
Germany
{'Tourism': {'R2': -0.7636445261419189, 'MSE': 0.029254332011986232}}
Greece
{'Tourism': {'R2': -0.2522715082007796, 'MSE': 1.2151193011355944e-05}}
Hungary
{'Tourism': {'R2': -0.5284532777804045, 'MSE': 0.0002666521954733702}}
Ireland
{'Tourism': {'R2': -1.6871670898184417, 'MSE

In [88]:
X = country_df[['Bias', 'Tourism_lag']].values
y_tourism = country_df['Tourism St.'].values
y_tourism

array([0.18251932, 0.16946152, 0.16967678, 0.1867815 , 0.19898867,
       0.27140702])